We will learn to read VCF files within R using a publicly available dataset of genomic variant calls for the infamous individual, NA12878. The Genome-in-a-Bottle Consortium has compiled consensus variant calls on this individual's genome and released this data for researchers to use. One of the main purposes of this data is to provide a benchmark for those to develop computational tools and analysis of human genomes. See https://github.com/genome-in-a-bottle/giab_latest_release

Variant Call Format (VCF) is a very common format for representing genomic variation data. See Lecture 16: Slides 19.

## 0. Install and load the `VariantAnnotation` Bioconductor package 
Load the `VariantAnnotation` package

In [1]:
#Sys.unsetenv("DISPLAY")

suppressPackageStartupMessages({
    #library(tidyverse)
    #library(GenomicRanges)
    #library(plyranges)
    library(VariantAnnotation)
    #library(Rsamtools)
})

## 1. Prepare parameters for reading VCF file.
There are a lot of variants in this file `GIAB_highconf_v.3.3.2.vcf.gz`, so we want to restrict to a smaller region for this example. 

### a. Setup parameters for scanning the VCF file.
First, we need to set up a `ScanVcfParam` object to read within `17:35500000-36000000`.

In [3]:
#old wdir
getwd()

#new wdir
data_dir = 'data'
setwd(data_dir)
getwd()

[1] "/fh/fast/henikoff_s/user/jgreene/projects/sciCnT/code"

[1] "/fh/fast/henikoff_s/user/jgreene/projects/TFCB/data"

In [4]:
list.files()

[1] "BRCA_IDC_cfDNA.bam"                           
[2] "BRCA_IDC_cfDNA.bam.bai"                       
[3] "BRCA.genome_wide_snp_6_broad_Level_3_scna.seg"
[4] "drive-download-20221115T203016Z-003.zip"      
[5] "GIAB_highconf_v.3.3.2.vcf.gz"                 
[6] "GIAB_highconf_v.3.3.2.vcf.gz.tbi"             
[7] "hs37d5-002.fa"                                
[8] "hs37d5.fa.gz.fai"

In [5]:
vcfFile <- "GIAB_highconf_v.3.3.2.vcf.gz"
vcfHead <- scanVcfHeader(vcfFile)
myGRange4 <- GRanges(seqnames = "17", ranges = IRanges(start = 35500000, end = 36000000))
vcf.param <- ScanVcfParam(which = myGRange4) 

## 2. Read the VCF file.

In [6]:
vcf <- readVcf(vcfFile, genome = "hg19", param = vcf.param)

The `vcf` variable is of class `CollapsedVCF` and will contain header information and data. Let's see what information has been parsed by `readVcf`.

## 3. Extract the contents of the VCF entries.

### a. Return the variants in this region as a `GRanges` object.
The `rowRanges` function will return a `GRanges` object containing the coordinates, REF/ALT bases, quality, and filtering status of the variants.

In [7]:
rowRanges(vcf)

GRanges object with 332 ranges and 5 metadata columns:
              seqnames            ranges strand | paramRangeID            REF
                 <Rle>         <IRanges>  <Rle> |     <factor> <DNAStringSet>
    rs2411161       17          35501799      * |           NA              C
    rs8073074       17          35502949      * |           NA              A
    rs4523972       17          35507230      * |           NA              C
  rs111498996       17 35507465-35507466      * |           NA             CA
    rs8077266       17          35509302      * |           NA              A
          ...      ...               ...    ... .          ...            ...
    rs8080225       17          35996195      * |           NA              T
    rs8075378       17          35996582      * |           NA              G
    rs6607281       17          35997126      * |           NA              T
    rs4332783       17          35997674      * |           NA              A
   rs7198

### b. Inspect the header information
The `INFO` column in the original VCF text file contains a semi-colon delimited set of custom fields with flexible format that algorithms will output.  Here, it is parsed into usable format. First, let's look at what fields are available from the header.

In [8]:
info(vcf) # returns a DataFrame object

DataFrame with 332 rows and 16 columns
                DPSum platforms       platformnames    platformbias  datasets
            <integer> <integer>     <CharacterList> <CharacterList> <integer>
rs2411161          NA         3   Illumina,CG,Solid                         3
rs8073074          NA         3   Illumina,CG,Solid                         4
rs4523972          NA         3   Illumina,CG,Solid                         3
rs111498996        NA         1            Illumina                         1
rs8077266          NA         3   Illumina,CG,Solid                         4
...               ...       ...                 ...             ...       ...
rs8080225          NA         4 Illumina,CG,10X,...                         4
rs8075378          NA         4 Illumina,CG,10X,...                         5
rs6607281          NA         4 Illumina,CG,10X,...                         4
rs4332783          NA         3     Illumina,CG,10X                         3
rs71984199         NA    

The `FORMAT` column in the original VCF text file contains the format and description of the genotype fields. Let's see what these are.

In [9]:
geno(header(vcf))

DataFrame with 8 rows and 3 columns
           Number        Type            Description
      <character> <character>            <character>
GT              1      String Consensus Genotype a..
DP              1     Integer Total read depth sum..
GQ              1     Integer Net Genotype quality..
ADALL           R     Integer Net allele depths ac..
AD              R     Integer Net allele depths ac..
IGT             1      String Original input genot..
IPS             1      String      Phase set for IGT
PS              1      String       Phase set for GT

### c. Inspect the genotype, read depth, and allele depth inforation.
To see the genotype `GT`, read depth `DP`, and allele depth `AD`, we access the the list.

In [10]:
geno(vcf)$GT[1:5]
geno(vcf)$DP[1:5]
geno(vcf)$AD[1:5]

[1] "1|1" "1|1" "1|1" "1|1" "1|1"

[1] 675 607 528 470 718

[[1]]
[1]  95 372

[[2]]
[1]  77 334

[[3]]
[1]  66 292

[[4]]
[1]   0 223

[[5]]
[1]  97 393

### d. Combine all `geno` fields into a single table.
You can also combine all fields into a `data.frame` object. But this code only works if the VCF contains a single sample.

In [11]:
genoData <- data.frame(do.call(cbind, geno(vcf)))
colnames(genoData) <- rownames(geno(header(vcf)))
genoData[1:10,]

,GT,DP,GQ,ADALL,AD,IGT,IPS,PS
,<named list>,<named list>,<named list>,<named list>,<named list>,<named list>,<named list>,<named list>
rs2411161,1|1,675,405,"1, 301","95, 372",1/1,.,PATMAT
rs8073074,1|1,607,387,"0, 278","77, 334",1/1,.,PATMAT
rs4523972,1|1,528,290,"1, 230","66, 292",1/1,.,PATMAT
rs111498996,1|1,470,99,"0, 223","0, 223",1/1,.,PATMAT
rs8077266,1|1,718,452,"0, 328","97, 393",1/1,.,PATMAT
rs75773263,1|1,773,233,"0, 378","31, 31",1/1,.,PATMAT
rs725038,1|1,896,287,"0, 442","51, 51",1/1,.,PATMAT
rs10664252,1|1,577,366,"0, 265","68, 68",1/1,.,PATMAT
rs11655013,0|1,720,1197,"159, 155","206, 193",0/1,.,PATMAT


## Exercise 3: Reading variants from a VCF file.

### a. Create a range for `8:128747680-128753680`.

In [12]:
# GRanges()

### b. Setup parameters to read VCF.

In [13]:
# ScanVcfParam

### c. Read the VCF file at `8:128747680-128753680`

In [14]:
# readVcf

### d. What is the RS id, genotype (`GT`) and depth (`DP`) at the SNP in this locus?

In [15]:
# geno()